In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
import gzip
from matplotlib import rc
import warnings

plt.style.use('seaborn-v0_8-white')
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
warnings.filterwarnings("ignore")
def load_data(filename):
    with gzip.open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

# Example usage
loaded_data = load_data('../figures/nfkb_arnold64by64.pkl.gz')
Omegas_tested = loaded_data["Omega"]
Coupling_strengths_tested = loaded_data["coupling_strength"]
A_ext_tested = loaded_data["A_ext"]
Entrainments = loaded_data["entrainment_value"]
Exceptions = loaded_data["exception"]
Ratios = np.array(loaded_data["calculated_ratio"])
Trajectories = loaded_data["trajectory"]
# for trajectory in Trajectories:
#     print(len(trajectory["t"]))
time = Trajectories[50]["t"]
n = Trajectories[50]["N_n"]
tnf = Trajectories[50]["TNF"]
omega = Omegas_tested[50]
omega
plt.hist(Ratios, bins=50)
plt.plot(time, n)
plt.plot(time, tnf)
from scipy.signal import find_peaks
peaks = find_peaks(tnf)
len(peaks[0])
plt.figure(figsize=(12,8), dpi=100)
plt.hist(Ratios.ravel(), bins=100)

plt.title("Histogram over beregnede Omega = T_ext/T_int uden afrunding, bins = 10")
plt.plot()
plt.figure(figsize=(12,8), dpi=100)
plt.hist(Ratios, bins=50)

plt.title("Histogram over beregnede Omega = T_ext/T_int uden afrunding, bins = 50")
plt.plot()
color_map = [
    # 'C7' if ratio - 1e-2 <= 1/4 <= ratio + 1e-2 else
    # 'C3' if ratio - 1e-2 <= 1/3 <= ratio + 1e-2 else
    'blue' if ratio - 1e-2 <= 2/4 <= ratio + 1e-2 else
    # 'C5' if ratio - 1e-2 <= 3/4 <= ratio + 1e-2 else
    'green' if ratio - 1e-2 <= 1/1 <= ratio + 1e-2 else
    # 'orange' if ratio - 1e-2 <= 3/2 <= ratio + 1e-2 else
    # 'C9' if ratio - 1e-2 <= 1.97 <= ratio + 1e-2 else
    'orange' if ratio - 1e-2 <= 4/2 <= ratio + 1e-2 else
    # 'C4' if ratio - 1e-2 <= 5/2 <= ratio + 1e-2 else
    # 'C9' if ratio - 1e-2 <= 6/2 <= ratio + 1e-2 else
    'red' if Exceptions[i] == True else
    'gray'
    for i, ratio in enumerate(Ratios.ravel())
]
plt.figure(figsize=(14,8))

plt.scatter(Omegas_tested, Coupling_strengths_tested, c=color_map, s=20, marker="s")

plt.title("Arnold Tongues", fontsize=20)
plt.ylabel("Coupling Strength", fontsize=16)
plt.xlabel("External / Internal Period", fontsize=16)
threshold = 0.005
tunger = [0, 0.5, 1, 1.5, 2, 2.5, 3]
tunger_v2 = np.linspace(0.1,3,30)
# zero_to_one = np.linspace(0.1,0.9,9)
# one_to_two = np.linspace(1.1,1.9,9)
# two_to_three = np.linspace(2.1,2.9,9)
# tunger_v3 = np.concatenate((zero_to_one, one_to_two, two_to_three))
# tunger_v3 = [0.2, 0.25, 0.3, 0.7, 0.75, 0.8, 1.2, 1.25, 1.3, 1.7, 1.75, 1.8, 2.2, 2.25, 2.3, 2.7, 2.75, 2.8,]

colors = ['C0', 'C1', 'C2', 'C6', 'C7', 'C8', 'C9']
# colors_v2 = ["red", "orange", "blue", "magenta", "green", "cyan", "darkviolet"]
colors_v3 = ["#DE1A1A", "#F1E1B6", "#DCCBD7", "#A2A3B9", "#CEE3EC", "#A488D4", "#80B8A7"]

def get_color_map(Entrainments, tunger, threshold, colors):
    color_map = []

    for entrainment in Entrainments:
        # Initialize the color to white (default color)
        color = 'white'

        # Check if entrainment falls within any of the specified ranges
        for i, start in enumerate(tunger_v2):
            if start - threshold <= entrainment <= start + threshold:
                color = '#d3d3d3'
                break  # Break out of the loop once a matching range is found

        # Check if entrainment falls within any of the specified ranges
        for i, start in enumerate(tunger):
            if start - threshold <= entrainment <= start + threshold:
                color = colors_v3[i]
                break  # Break out of the loop once a matching range is found

        color_map.append(color)

    return color_map

color_map = get_color_map(Ratios, tunger, threshold, colors)
# Create a figure and axis explicitly
fig, ax = plt.subplots(figsize=(16,9), dpi=800)

ax.scatter(Omegas_tested, Coupling_strengths_tested, c=color_map, marker='o', s=10)

# Scatter plot
scatter = plt.scatter(Omegas_tested, Coupling_strengths_tested, c=color_map, marker='o', s=10)

# Title and labels
ax.set_title("Arnold Tongues", fontsize=20)
ax.set_ylabel("Coupling Strength", fontsize=16)
ax.set_xlabel("External / Internal Period", fontsize=16)

# Adjust the width of the color strips by defining the number of surrounding indices to color
strip_width = 1 # Increase or decrease to adjust the width of the color strip

# Manually create a colormap with wider strips
cmap_list = ['white'] * 256  # Start with all white
for i, color in enumerate(colors_v3):
    index = int(tunger[i] / 3.0 * 255)  # Map the tunger value to 0-255
    # Apply color to a range of indices around the central index
    for j in range(max(0, index - strip_width), min(256, index + strip_width + 1)):
        cmap_list[j] = color
custom_cmap = mpl.colors.ListedColormap(cmap_list)

# Normalization from 0 to 3
norm = mpl.colors.Normalize(vmin=0, vmax=3)

# Create a colorbar with the specified cmap and norm
sm = mpl.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, ticks=tunger)
cbar.set_label('Entrainment Ratio', fontsize=16)

# Show the plot
plt.show()
# plt.title("Arnold Tongues", fontsize=20)
# plt.ylabel("Coupling Strength", fontsize=16)
# plt.xlabel("External / Internal Period", fontsize=16)
# plt.colorbar()
# plt.show()
